<a href="https://colab.research.google.com/github/tonystark11/transformer-from-scratch/blob/main/src/translation_transformer_from_scratch_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [44]:
# !pip install -U 'spacy[cuda-autodetect]' -q

In [45]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download es_core_news_sm

# Imports

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
from functools import partial
from tqdm import tqdm

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [47]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [48]:
random_seed = 42

# MultiHeadAttention

In [49]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_len, d_model = x.size()
        return x.reshape(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_len, d_k = x.size()
        return x.transpose(1, 2).reshape(batch_size, seq_len, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output


# Position wise Feed Forward Network

In [50]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

# Positional Encoding

In [51]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model, device=device)
        position = torch.arange(0, max_seq_length, dtype=torch.float, device=device).unsqueeze(1)
        div_term = torch.pow(10_000, (-torch.arange(0, d_model, 2, device=device).float() / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))


    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# Encoder Layer

In [52]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x


# Decoder Layer

In [53]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)


    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

# Transformer Model

In [54]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length, device=device), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)

        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

# Load Data

In [55]:
# !wget https://www.manythings.org/anki/spa-eng.zip

In [56]:
# !unzip spa-eng.zip

In [57]:
with open('spa.txt', 'r') as f:
    lines = f.readlines()

In [58]:
len(lines)

141543

In [59]:
lines[10000]

"I can't see you.\tNo puedo veros.\tCC-BY 2.0 (France) Attribution: tatoeba.org #1784974 (Amastan) & #1784978 (Shishir)\n"

In [60]:
# Remove everything after the 2nd tab character.
# As we can see above, we only need the first two columns of the data
lines = [line.split('\t') for line in lines]
lines = ['\t'.join(line[:2]) for line in lines]

In [61]:
lines[10000]

"I can't see you.\tNo puedo veros."

In [62]:
# Create train, val, test split
train_lines, val_test_lines = train_test_split(lines, test_size=0.2, random_state=random_seed, shuffle=True)
val_lines, test_lines = train_test_split(val_test_lines, test_size=0.5, random_state=random_seed, shuffle=True)

In [63]:
print(len(train_lines))
print(len(val_lines))
print(len(test_lines))

113234
14154
14155


In [64]:
train_lines[0]

"Why didn't you use a pay phone?\t¿Por qué no usaste un teléfono de pago?"

In [65]:
val_lines[0]

'Tom advised Mary to go to see a doctor.\tTom le aconsejó a Mary que vaya a ver al doctor.'

In [66]:
test_lines[0]

"Tom's flight was delayed.\tEl vuelo de Tom fue retrasado."

# Preprocess Data

In [67]:
SRC_LANGUAGE = "en"
TGT_LANGUAGE = "es"

In [68]:
tokenizer = {}
tokenizer[SRC_LANGUAGE] = get_tokenizer("spacy", "en_core_web_sm")
tokenizer[TGT_LANGUAGE] = get_tokenizer("spacy", "es_core_news_sm")

## Create Dataset

In [69]:
class SentencePairDataset(Dataset):
    def __init__(self, lines, src_tokenizer, tgt_tokenizer):
        super(SentencePairDataset, self).__init__()

        self.lines = lines
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        line = self.lines[idx]

        src, tgt = line.split('\t')
        src_tokens = self.src_tokenizer(src)
        tgt_tokens = self.tgt_tokenizer(tgt)

        return src_tokens, tgt_tokens

In [70]:
train_ds = SentencePairDataset(train_lines, tokenizer[SRC_LANGUAGE], tokenizer[TGT_LANGUAGE])
val_ds = SentencePairDataset(val_lines, tokenizer[SRC_LANGUAGE], tokenizer[TGT_LANGUAGE])
test_ds = SentencePairDataset(test_lines, tokenizer[SRC_LANGUAGE], tokenizer[TGT_LANGUAGE])

In [71]:
# Length of longest src sequence
print(max(len(x[0]) for x in train_ds))
print(max(len(x[0]) for x in val_ds))
print(max(len(x[0]) for x in test_ds))

81
46
32


In [72]:
# Length of longest tgt sequence
print(max(len(x[1]) for x in train_ds))
print(max(len(x[1]) for x in val_ds))
print(max(len(x[1]) for x in test_ds))

78
44
34


In [73]:
next(iter(train_ds))

(['Why', 'did', "n't", 'you', 'use', 'a', 'pay', 'phone', '?'],
 ['¿', 'Por', 'qué', 'no', 'usaste', 'un', 'teléfono', 'de', 'pago', '?'])

## Create Vocabulary

In [74]:
vocab = {}

In [75]:
src_vocab_size = 10_000
tgt_vocab_size = 10_000
max_seq_len = 100

PAD_IDX = 0
UNK_IDX = 1
BOS_IDX = 2
EOS_IDX = 3

special_symbols = ['<PAD>', '<UNK>', '<BOS>', '<EOS>']

In [76]:
def yield_tokens(dataset, lang_idx=0):
    n = len(dataset)
    i = 0

    while i < n:
        yield dataset[i][lang_idx]
        i += 1

In [77]:
src_iterator = yield_tokens(train_ds, lang_idx=0)
tgt_iterator = yield_tokens(train_ds, lang_idx=1)

In [78]:
vocab[SRC_LANGUAGE] = build_vocab_from_iterator(
    src_iterator,
    min_freq=1,
    specials=special_symbols,
    special_first=True,
    max_tokens=src_vocab_size,
)

In [79]:
vocab[TGT_LANGUAGE] = build_vocab_from_iterator(
    tgt_iterator,
    min_freq=1,
    specials=special_symbols,
    special_first=True,
    max_tokens=tgt_vocab_size,
)

In [80]:
vocab[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab[TGT_LANGUAGE].set_default_index(UNK_IDX)

In [81]:
vocab[SRC_LANGUAGE]['hello']

2325

In [82]:
vocab[TGT_LANGUAGE]['Hola']

2337

In [83]:
def collate_fn(batch, vocab):
    batch_size = len(batch)
    srcs, tgts = zip(*batch)
    src_vectors = torch.zeros((batch_size, max_seq_len), dtype=torch.long, device=device)
    tgt_vectors = torch.zeros((batch_size, max_seq_len), dtype=torch.long, device=device)

    for i in range(batch_size):
        src_vectors[i] = torch.tensor(([BOS_IDX] + vocab[SRC_LANGUAGE](srcs[i]) + [EOS_IDX] + [0] * (max_seq_len - len(srcs[i])))[:max_seq_len], dtype=torch.long, device=device)
        tgt_vectors[i] = torch.tensor(([BOS_IDX] + vocab[TGT_LANGUAGE](tgts[i]) + [EOS_IDX] + [0] * (max_seq_len - len(tgts[i])))[:max_seq_len], dtype=torch.long, device=device)

    return src_vectors, tgt_vectors

In [84]:
train_dataloader = DataLoader(train_ds, batch_size=64, shuffle=True, collate_fn=partial(collate_fn, vocab=vocab))
val_dataloader = DataLoader(val_ds, batch_size=64, shuffle=True, collate_fn=partial(collate_fn, vocab=vocab))
test_dataloader = DataLoader(test_ds, batch_size=64, shuffle=True, collate_fn=partial(collate_fn, vocab=vocab))

In [85]:
src_vocab_size = 10_000
tgt_vocab_size = 10_000
d_model = 512
num_heads = 4
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1
num_epochs = 3

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(num_epochs):
    print(f"Epoch: {epoch+1}\n------------------------------")
    transformer.train()
    train_loss = 0
    for data in tqdm(train_dataloader, total=len(train_dataloader)):
        src_data, tgt_data = data
        optimizer.zero_grad()
        output = transformer(src_data, tgt_data[:, :-1])
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    transformer.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_dataloader:
            src_data, tgt_data = data
            output = transformer(src_data, tgt_data[:, :-1])
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
            val_loss += loss.item()
    
    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)
            
    print(f"Epoch: {epoch+1} | Training Loss: {train_loss} | Validation Loss: {val_loss}")
    # torch.save(transformer.state_dict(), f'./transformer_state_dict_epoch_{epoch+1}')

Epoch: 1
------------------------------


100%|██████████| 1770/1770 [06:18<00:00,  4.68it/s]


Epoch: 1 | Training Loss: 3.1201352871743975 | Validation Loss: 2.07179338384319
Epoch: 2
------------------------------


100%|██████████| 1770/1770 [05:19<00:00,  5.53it/s]


Epoch: 2 | Training Loss: 1.9061797309730013 | Validation Loss: 1.6167761028349936
Epoch: 3
------------------------------


100%|██████████| 1770/1770 [05:21<00:00,  5.51it/s]


Epoch: 3 | Training Loss: 1.5077081775261183 | Validation Loss: 1.4094878392176584


In [86]:
transformer.eval()
with torch.no_grad():
    test_loss = 0
    for data in tqdm(test_dataloader, total=len(test_dataloader)):
        src_data, tgt_data = data
        output = transformer(src_data, tgt_data[:, :-1])
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
        test_loss += loss.item()
    test_loss /= len(test_dataloader)
    print(f"Test Loss: {test_loss}")

100%|██████████| 222/222 [00:14<00:00, 15.60it/s]

Test Loss: 1.4262425480662166


In [98]:
len(train_dataloader) * 64, len(val_dataloader) * 64

(113280, 14208)

# Inference

In [88]:
# model_path = "./transformer_epoch_3"
# state_dict = torch.load(model_path)

src_vocab_size = 10_000
tgt_vocab_size = 10_000
d_model = 512
num_heads = 4
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1
num_epochs = 3

# transformer_loaded = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)
# transformer_loaded.load_state_dict(state_dict)

In [89]:
tokenizer[SRC_LANGUAGE]('Hello, i am a teacher')

['Hello', ',', 'i', 'am', 'a', 'teacher']

In [94]:
def translate(src):
    src_tokens = tokenizer[SRC_LANGUAGE](src)
    tgt_tokens = ["<BOS>"]

    src_vectors = torch.tensor(([BOS_IDX] + vocab[SRC_LANGUAGE](src_tokens) + [EOS_IDX] + [0] * (max_seq_len - len(src_tokens)))[:max_seq_len], dtype=torch.long, device=device).unsqueeze(0)

    for i in range(max_seq_len):
        tgt_vectors = torch.tensor((vocab[TGT_LANGUAGE](tgt_tokens) + [0] * (max_seq_len - len(tgt_tokens)))[:max_seq_len], dtype=torch.long, device=device).unsqueeze(0)
        output = transformer(src_vectors, tgt_vectors)
        idx = torch.argmax(nn.functional.softmax(output, dim=2)[0][i]).item()
        tgt_tokens.append(vocab[TGT_LANGUAGE].lookup_token(idx))

        if idx == EOS_IDX:
            break

    return " ".join(tgt_tokens).replace("<BOS>", "").replace("<EOS>", "").replace("<PAD>", "").strip()

In [95]:
translate("Hello, I am a teacher.")

'Hola , soy profesor .'

In [48]:
translate("My name is John.")

'Mi nombre es John .'

In [49]:
translate("I am learning Spanish.")

'Estoy aprendiendo español .'

In [50]:
translate("I eat apples.")

'Yo como manzanas .'

In [51]:
translate("I have three books and two pens.")

'Tengo tres libros y dos <UNK> .'

In [52]:
translate("Do you work in an office?")

'¿ Tú trabajo en una oficina ?'

In [53]:
translate("How are you?")

'¿ Cómo estás ?'

In [54]:
eng, spa = test_lines[0].split('\t')
print(eng)
print(spa)
translate(eng)

Tom's flight was delayed.
El vuelo de Tom fue retrasado.


'El vuelo de Tom estaba <UNK> .'

In [55]:
eng, spa = test_lines[500].split('\t')
print(eng)
print(spa)
translate(eng)

I can't believe Tom really said no to me.
No puedo creer que Tom verdaderamente me haya dicho no.


'No puedo creer que Tom realmente dijo que no me dijo .'

In [56]:
eng, spa = train_lines[1000].split('\t')
print(eng)
print(spa)
translate(eng)

Tom eats breakfast every morning at six o'clock.
Tom desayuna a las seis todas las mañanas.


'Tom come todos los días a las seis .'

In [57]:
eng, spa = train_lines[10000].split('\t')
print(eng)
print(spa)
translate(eng)

He can speak five languages.
Él habla cinco lenguas.


'Él sabe hablar cinco idiomas .'

# Export model and vocabulary

In [197]:
torch.save(vocab[SRC_LANGUAGE], "./vocab-english")
torch.save(vocab[TGT_LANGUAGE], "./vocab-spanish")

In [203]:
torch.save(tokenizer[SRC_LANGUAGE], "./tokenizer-english")
torch.save(tokenizer[TGT_LANGUAGE], "./tokenizer-spanish")

In [210]:
torch.save(transformer, "./transformer_model")